<a href="https://colab.research.google.com/github/schatz06/Thesis/blob/main/Notebooks_CNN_HFO_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Mount drive## 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##General Variables to use to load data##

In [2]:
fold = 2 # which fold to take 
embedding = "protbert"
# embedding = "seqvec"
# dataset="PISCES" 
dataset="CB513"
USE_HFO=True

## Imports ##

In [3]:
# Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
%load_ext autoreload 
%autoreload 2
# matplotlib graphs will be included in your notebook, next to the code. 
%matplotlib inline

In [4]:
# install hdf5storage package 
!pip install hdf5storage

In [5]:
import pdb # python debugger
import numpy as np # import numpy 
import tensorflow as tf # import tensorflow  
tf.compat.v1.disable_eager_execution() # disable eager execution
import time # import time
import math # import math
import argparse # import argparse
import os # import os
import scipy.io as sio # import scipy.io 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() # makes different behaviors betweem tf_v1 & tf_v2 behave the same 
from tensorflow.python.client import device_lib # package to find available gpus
import pandas as pd # import padas
import hdf5storage # import hdf5storage

Instructions for updating:
non-resource variables are not supported in the long term


## Get data ##

In [6]:
VALID_FILE = "/content/drive/MyDrive/Datasets/{0}_{1}_testSet{2}.mat".format(dataset.lower(),embedding,str(fold)) # validation set
TRAIN_FILE = "/content/drive/MyDrive/Datasets/{0}_{1}_trainSet{2}.mat".format(dataset.lower(),embedding,str(fold)) # train set  
TEST_FILE = "/content/drive/MyDrive/Datasets/casp13_{0}.mat".format(embedding) # test set CASP13
print(VALID_FILE)
print(TRAIN_FILE)
print(TEST_FILE)

/content/drive/MyDrive/Datasets/cb513_protbert_testSet2.mat
/content/drive/MyDrive/Datasets/cb513_protbert_trainSet2.mat
/content/drive/MyDrive/Datasets/casp13_protbert.mat


In [7]:
HEIGHT = 32
WIDTH = 32
DEPTH = 1
CATEGORIES = 3 # number of different classification categories

## VGG ##

In [8]:
"""
Codes are modifeid from PyTorch and Tensorflow Versions of VGG: 
https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py, and
https://github.com/keras-team/keras-applications/blob/master/keras_applications/vgg16.py
"""

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import numpy as np 
# import pdb
from tensorflow.keras.applications.vgg16 import VGG16 as vgg16 # import vgg16 convolutional network
from tensorflow.keras.applications.vgg19 import VGG19 as vgg19 # import vgg19 convolutional network

__all__ = ['VGG11', 'VGG13', 'VGG16','VGG19'] # array holds all vgg CNN's names
 
def VGG(feature, num_cls): # define VGG 

	with tf.variable_scope('fully_connected') as scope:
		dim =np.prod(feature.shape[1:]) # returns the product of the given array
		x = tf.reshape(feature, [-1, dim]) # reshape tensor

		x = tf.keras.layers.Dense(units=4096, activation='relu', name=scope.name)(x) # define layers 
		x = tf.keras.layers.Dense(units=4096, activation='relu', name=scope.name)(x)
		x = tf.keras.layers.Dense(units=num_cls, name=scope.name)(x)

	return x
# make the layers of CNN 
def make_layers(x, cfg):
	for v in cfg:
		if v == 'M':
			x = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='valid')(x)
		else:
			x = tf.keras.layers.Conv2D(
			filters=v,
			kernel_size=[3, 3],
			padding='SAME',
			activation=tf.nn.relu
			)(x)
	return x

cfg = {
	'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
	'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
	'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
	'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 
		  512, 512, 512, 512, 'M'],
}

def VGG11(x_images, num_cls):
	feature = make_layers(x_images, cfg['A'])
	return VGG(feature, num_cls)

def VGG13(x_images, num_cls):
	feature = make_layers(x_images, cfg['B'])
	return VGG(feature, num_cls)

def VGG16(x_images, num_cls):
	feature = make_layers(x_images, cfg['D'])
	return VGG(feature, num_cls)

def VGG19(x_images, num_cls):
	feature = make_layers(x_images, cfg['E'])
	return VGG(feature, num_cls)

## Net ##

In [9]:
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import math
# import pdb
# from tensorflow.python.client import device_lib
# import numpy as np
# from net.vgg import *

def CNN_4layers(x_image, num_cls, reuse=False):
	_NUM_CLASSES = num_cls
	with tf.variable_scope('conv1', reuse=reuse) as scope:
		conv = tf.keras.layers.Conv2D(
			filters=64,
			kernel_size=[3, 3],
			padding='SAME',
			activation=tf.nn.relu
		)(x_image)

	with tf.variable_scope('conv2', reuse=reuse) as scope:
		conv = tf.keras.layers.Conv2D(
			filters=64,
			kernel_size=[3, 3],
			padding='SAME',
			activation=tf.nn.relu
		)(conv)
		
	with tf.variable_scope('conv3', reuse=reuse) as scope:
		conv = tf.keras.layers.Conv2D(
			filters=64,
			kernel_size=[3, 3],
			padding='SAME',
			activation=tf.nn.relu
		)(conv)

	with tf.variable_scope('fully_connected', reuse=reuse) as scope:
		dim =np.prod(conv.shape[1:])
		flat = tf.reshape(conv, [-1, dim])
		outputs = tf.keras.layers.Dense(units=_NUM_CLASSES, name=scope.name)(flat)

	return outputs


	# with tf.variable_scope('conv1', reuse=reuse) as scope:
	# 	conv = tf.keras.layers.Conv2D(
	# 		filters=32,
	# 		kernel_size=[5, 5],
	# 		padding='SAME',
	# 		activation=tf.nn.relu
	# 	)(x_image)
	# 	pool = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='valid')(conv)
	# 	# N x 16 x 16 x 32

	# with tf.variable_scope('conv2', reuse=reuse) as scope:
	# 	conv = tf.keras.layers.Conv2D(
	# 		filters=64,
	# 		kernel_size=[3, 3],
	# 		padding='SAME',
	# 		activation=tf.nn.relu
	# 	)(pool)
	# 	pool = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='valid')(conv)
	# 	# N x 8 x 8 x 64
		
	# with tf.variable_scope('conv3', reuse=reuse) as scope:
	# 	conv = tf.keras.layers.Conv2D(
	# 		filters=64,
	# 		kernel_size=[3, 3],
	# 		padding='SAME',
	# 		activation=tf.nn.relu
	# 	)(pool)
	# 	pool = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='valid')(conv)
	# 	# N x 4 x 4 x 64

	# with tf.variable_scope('fully_connected', reuse=reuse) as scope:
	# 	dim =np.prod(pool.shape[1:])
	# 	flat = tf.reshape(pool, [-1, dim])
	# 	outputs = tf.keras.layers.Dense(units=_NUM_CLASSES, name=scope.name)(flat)

	# return outputs

def CNN_7layers(x_image, num_cls, reuse=False):
	_NUM_CLASSES = num_cls
	with tf.variable_scope('conv1', reuse=reuse) as scope:
		conv = tf.keras.layers.Conv2D(
			filters=64,
			kernel_size=[3, 3],
			padding='SAME',
			activation=tf.nn.relu
		)(x_image)
		conv = tf.keras.layers.Conv2D(
			filters=64,
			kernel_size=[3, 3],
			padding='SAME',
			activation=tf.nn.relu
		)(conv)

	with tf.variable_scope('conv2', reuse=reuse) as scope:
		conv = tf.keras.layers.Conv2D(
			filters=64,
			kernel_size=[3, 3],
			padding='SAME',
			activation=tf.nn.relu
		)(conv)
		conv = tf.keras.layers.Conv2D(
			filters=64,
			kernel_size=[3, 3],
			padding='SAME',
			activation=tf.nn.relu
		)(conv)

	with tf.variable_scope('conv3', reuse=reuse) as scope:
		conv = tf.keras.layers.Conv2D(
			filters=64,
			kernel_size=[3, 3],
			padding='SAME',
			activation=tf.nn.relu
		)(conv)
		conv = tf.keras.layers.Conv2D(
			filters=64,
			kernel_size=[3, 3],
			padding='SAME',
			activation=tf.nn.relu
		)(conv)

	with tf.variable_scope('fully_connected', reuse=reuse) as scope:
		dim = np.prod(conv.shape[1:])
		flat = tf.reshape(conv, [-1, dim])
		outputs = tf.keras.layers.Dense(units=_NUM_CLASSES, name=scope.name)(flat)

	return outputs

	# with tf.variable_scope('conv1', reuse=reuse) as scope:
	# 	conv = tf.keras.layers.Conv2D(
	# 		filters=32,
	# 		kernel_size=[5, 5],
	# 		padding='SAME',
	# 		activation=tf.nn.relu
	# 	)(x_image)
	# 	conv = tf.keras.layers.Conv2D(
	# 		filters=32,
	# 		kernel_size=[3, 3],
	# 		padding='SAME',
	# 		activation=tf.nn.relu
	# 	)(conv)
	# 	pool = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='valid')(conv)
	# 	# N x 16 x 16 x 32

	# with tf.variable_scope('conv2', reuse=reuse) as scope:
	# 	conv = tf.keras.layers.Conv2D(
	# 		filters=64,
	# 		kernel_size=[3, 3],
	# 		padding='SAME',
	# 		activation=tf.nn.relu
	# 	)(pool)
	# 	conv = tf.keras.layers.Conv2D(
	# 		filters=64,
	# 		kernel_size=[3, 3],
	# 		padding='SAME',
	# 		activation=tf.nn.relu
	# 	)(conv)
	# 	pool = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='valid')(conv)
	# 	# N x 8 x 8 x 64

	# with tf.variable_scope('conv3', reuse=reuse) as scope:
	# 	conv = tf.keras.layers.Conv2D(
	# 		filters=64,
	# 		kernel_size=[3, 3],
	# 		padding='SAME',
	# 		activation=tf.nn.relu
	# 	)(pool)
	# 	conv = tf.keras.layers.Conv2D(
	# 		filters=128,
	# 		kernel_size=[3, 3],
	# 		padding='SAME',
	# 		activation=tf.nn.relu
	# 	)(conv)
	# 	pool = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='valid')(conv)
	# 	# pool = tf.layers.dropout(pool, rate=0.25, name=scope.name)
	# 	# N x 4 x 4 x 128

	# with tf.variable_scope('fully_connected', reuse=reuse) as scope:
	# 	dim = np.prod(pool.shape[1:])
	# 	flat = tf.reshape(pool, [-1, dim])
	# 	outputs = tf.keras.layers.Dense(units=_NUM_CLASSES, name=scope.name)(flat)

	# return outputs

def CNN(net, num_cls, dim):

	_NUM_CLASSES = num_cls
	_IMAGE_HEIGHT, _IMAGE_WIDTH, _IMAGE_CHANNELS = dim

	with tf.name_scope('main_params'):
		x = tf.placeholder(tf.float32, shape=[None, _IMAGE_HEIGHT, _IMAGE_WIDTH, _IMAGE_CHANNELS], name='input_of_net')
		y = tf.placeholder(tf.float32, shape=[None, _NUM_CLASSES], name='labels')

	# call CNN structure according to string net
	outputs = globals()[net](x, _NUM_CLASSES)
	outputs = tf.identity(outputs, name='output_of_net')

	return (x, y, outputs)

## Utilities ##

In [10]:
# import numpy as np
# import math
# import scipy.io as sio
# import os
# import math
# import pdb

class ConfigClass(object):
	def __init__(self, args, num_data, num_cls):
		super(ConfigClass, self).__init__()
		self.args = args
		self.iter_max = args.iter_max
		
		# Different notations of regularization term:
		# In SGD, weight decay:
		# 	weight_decay <- lr/(C*num_of_training_samples)
		# In Newton method:
		# 	C <- C * num_of_training_samples

		self.seed = args.seed

		if self.seed is None:
			print('You choose not to specify a random seed.'+\
				'A different result is produced after each run.')
		elif isinstance(self.seed, int) and self.seed >= 0:
			print('You specify random seed {}.'.format(self.seed))
		else:
			raise ValueError('Only accept None type or nonnegative integers for'+\
					' random seed argument!')

		self.train_set = args.train_set
		self.val_set = args.val_set
		self.num_cls = num_cls
		self.dim = args.dim

		self.num_data = num_data
		self.GNsize = min(args.GNsize, self.num_data)
		self.C = args.C * self.num_data
		self.net = args.net

		self.xi = 0.1
		self.CGmax = args.CGmax
		self._lambda = args._lambda
		self.drop = args.drop
		self.boost = args.boost
		self.eta = args.eta
		self.lr = args.lr
		self.lr_decay = args.lr_decay

		self.bsize = args.bsize
		if args.momentum < 0:
			raise ValueError('Momentum needs to be larger than 0!')
		self.momentum = args.momentum

		self.loss = args.loss
		if self.loss not in ('MSELoss', 'CrossEntropy'):
			raise ValueError('Unrecognized loss type!')
		self.optim = args.optim
		if self.optim not in ('SGD', 'NewtonCG', 'Adam'):
			raise ValueError('Only support SGD, Adam & NewtonCG optimizer!')
		
		self.log_file = args.log_file
		self.model_file = args.model_file
		self.screen_log_only = args.screen_log_only

		if self.screen_log_only:
			print('You choose not to store running log. Only store model to {}'.format(self.log_file))
		else:
			print('Saving log to: {}'.format(self.log_file))
			dir_name, _ = os.path.split(self.log_file)
			if not os.path.isdir(dir_name):
				os.makedirs(dir_name, exist_ok=True)

		dir_name, _ = os.path.split(self.model_file)
		if not os.path.isdir(dir_name):
			os.makedirs(dir_name, exist_ok=True)
		
		self.elapsed_time = 0.0

def read_data(filename, dim, label_enum=None):
	"""
	args:
		filename: the path where .mat files are stored
		label_enum (default None): the list that stores the original labels. 
			If label_enum is None, the function will generate a new list which stores the 
			original labels in a sequence, and map original labels to [0, 1, ... number_of_classes-1]. 
			If label_enum is a list, the function will use it to convert 
			original labels to [0, 1,..., number_of_classes-1].
	"""

	mat_contents = sio.loadmat(filename)
	#mat_contents = hdf5storage.loadmat(filename)
	images, labels = mat_contents['x'], mat_contents['y']
	
	labels = labels.reshape(-1)
	images = images.reshape(images.shape[0], -1)

	_IMAGE_HEIGHT, _IMAGE_WIDTH, _IMAGE_CHANNELS = dim
	zero_to_append = np.zeros((images.shape[0],
			_IMAGE_CHANNELS*_IMAGE_HEIGHT*_IMAGE_WIDTH-np.prod(images.shape[1:])))
	images = np.append(images, zero_to_append, axis=1)

	# check data validity
	if label_enum is None:
		label_enum, labels = np.unique(labels, return_inverse=True)
		num_cls = labels.max() + 1

		if len(label_enum) != num_cls:
			raise ValueError('The number of classes is not equal to the number of\
							labels in dataset. Please verify them.')
	else:
		num_cls = len(label_enum)
		forward_map = dict(zip(label_enum, np.arange(num_cls)))
		labels = np.expand_dims(labels, axis=1)
		labels = np.apply_along_axis(lambda x:forward_map[x[0]], axis=1, arr=labels)
		

	# convert groundtruth to one-hot encoding
	labels = np.eye(num_cls)[labels]
	labels = labels.astype('float32')

	return [images, labels], num_cls, label_enum

def normalize_and_reshape(images, dim, mean_tr=None):
	_IMAGE_HEIGHT, _IMAGE_WIDTH, _IMAGE_CHANNELS = dim
	images_shape = [images.shape[0], _IMAGE_CHANNELS, _IMAGE_HEIGHT, _IMAGE_WIDTH]

	# images normalization and zero centering
	images = images.reshape(images_shape[0], -1)

	images = images/255.0

	if mean_tr is None:
		print('No mean of data provided! Normalize images by their own mean.')
		# if no mean_tr is provided, we calculate it according to the current data
		mean_tr = images.mean(axis=0) 
	else:
		print('Normalize images according to the provided mean.')
		if np.prod(mean_tr.shape) != np.prod(dim):
			raise ValueError('Dimension of provided mean does not agree with the data! Please verify them!')

	images = images - mean_tr

	images = images.reshape(images_shape)
	# Tensorflow accepts data shape: B x H x W x C
	images = np.transpose(images, (0, 2, 3, 1))
	return images, mean_tr


def predict(sess, network, test_batch, bsize):
	x, y, loss, outputs = network

	test_inputs, test_labels = test_batch
	batch_size = bsize

	num_data = test_labels.shape[0]
	num_batches = math.ceil(num_data/batch_size)

	results = np.zeros(shape=num_data, dtype=np.int)
	infer_loss = 0.0

	for i in range(num_batches):
		batch_idx = np.arange(i*batch_size, min((i+1)*batch_size, num_data))

		batch_input = test_inputs[batch_idx]
		batch_labels = test_labels[batch_idx]

		net_outputs, _loss = sess.run(
			[outputs, loss], feed_dict={x: batch_input, y: batch_labels}
			)
		
		results[batch_idx] = np.argmax(net_outputs, axis=1)
		# note that _loss was summed over batches
		infer_loss = infer_loss + _loss

	avg_acc = (np.argmax(test_labels, axis=1) == results).mean()
	avg_loss = infer_loss/num_data
	
	return avg_loss, avg_acc, results

## Newton - CG ##

In [11]:
# import pdb
# import tensorflow as tf
# import time
# import numpy as np
# import os
# import math
# from utilities import predict

def Rop(f, weights, v):
	"""Implementation of R operator
	Args:
		f: any function of weights
		weights: list of tensors.
		v: vector for right multiplication
	Returns:
		Jv: Jaccobian vector product, length same as
			the number of output of f
	"""
	if type(f) == list:
		u = [tf.zeros_like(ff) for ff in f]
	else:
		u = tf.zeros_like(f)  # dummy variable
	g = tf.gradients(ys=f, xs=weights, grad_ys=u)
	return tf.gradients(ys=g, xs=u, grad_ys=v)

def Gauss_Newton_vec(outputs, loss, weights, v):
	"""Implements Gauss-Newton vector product.
	Args:
		loss: Loss function.
		outputs: outputs of the last layer (pre-softmax).
		weights: Weights, list of tensors.
		v: vector to be multiplied with Gauss Newton matrix
	Returns:
		J'BJv: Guass-Newton vector product.
	"""
	# Validate the input
	if type(weights) == list:
		if len(v) != len(weights):
			raise ValueError("weights and v must have the same length.")

	grads_outputs = tf.gradients(ys=loss, xs=outputs)
	BJv = Rop(grads_outputs, weights, v)
	JBJv = tf.gradients(ys=outputs, xs=weights, grad_ys=BJv)
	return JBJv
	

class newton_cg(object):
	def __init__(self, config, sess, outputs, loss):
		"""
		initialize operations and vairables that will be used in newton
		args:
			sess: tensorflow session
			outputs: output of the neural network (pre-softmax layer)
			loss: function to calculate loss
		"""
		super(newton_cg, self).__init__()
		self.sess = sess
		self.config = config
		self.outputs = outputs
		self.loss = loss
		self.param = tf.compat.v1.trainable_variables()

		self.CGiter = 0
		FLOAT = tf.float32
		model_weight = self.vectorize(self.param)
		
		# initial variable used in CG
		zeros = tf.zeros(model_weight.get_shape(), dtype=FLOAT)
		self.r = tf.Variable(zeros, dtype=FLOAT, trainable=False)
		self.v = tf.Variable(zeros, dtype=FLOAT, trainable=False)
		self.s = tf.Variable(zeros, dtype=FLOAT, trainable=False)
		self.g = tf.Variable(zeros, dtype=FLOAT, trainable=False)
		# initial Gv, f for method minibatch
		self.Gv = tf.Variable(zeros, dtype=FLOAT, trainable=False)
		self.f = tf.Variable(0., dtype=FLOAT, trainable=False)

		# rTr, cgtol and beta to be used in CG
		self.rTr = tf.Variable(0., dtype=FLOAT, trainable=False)
		self.cgtol = tf.Variable(0., dtype=FLOAT, trainable=False)
		self.beta = tf.Variable(0., dtype=FLOAT, trainable=False)

		# placeholder alpha, old_alpha and lambda
		self.alpha = tf.compat.v1.placeholder(FLOAT, shape=[])
		self.old_alpha = tf.compat.v1.placeholder(FLOAT, shape=[])
		self._lambda = tf.compat.v1.placeholder(FLOAT, shape=[])

		self.num_grad_segment = math.ceil(self.config.num_data/self.config.bsize)
		self.num_Gv_segment = math.ceil(self.config.GNsize/self.config.bsize)

		cal_loss, cal_lossgrad, cal_lossGv, \
		add_reg_avg_loss, add_reg_avg_grad, add_reg_avg_Gv, \
		zero_loss, zero_grad, zero_Gv = self._ops_in_minibatch()

		# initial operations that will be used in minibatch and newton
		self.cal_loss = cal_loss
		self.cal_lossgrad = cal_lossgrad
		self.cal_lossGv = cal_lossGv
		self.add_reg_avg_loss = add_reg_avg_loss
		self.add_reg_avg_grad = add_reg_avg_grad
		self.add_reg_avg_Gv = add_reg_avg_Gv
		self.zero_loss = zero_loss
		self.zero_grad = zero_grad
		self.zero_Gv = zero_Gv

		self.CG, self.update_v = self._CG()
		self.init_cg_vars = self._init_cg_vars()
		self.update_gs = tf.tensordot(self.s, self.g, axes=1)
		self.update_sGs = 0.5*tf.tensordot(self.s, -self.g-self.r-self._lambda*self.s, axes=1)
		self.update_model = self._update_model()
		self.gnorm = self.calc_norm(self.g)


	def vectorize(self, tensors):
		if isinstance(tensors, list) or isinstance(tensors, tuple):
			vector = [tf.reshape(tensor, [-1]) for tensor in tensors]
			return tf.concat(vector, 0) 
		else:
			return tensors 
	
	def inverse_vectorize(self, vector, param):
		if isinstance(vector, list):
			return vector
		else:
			tensors = []
			offset = 0
			num_total_param = np.sum([np.prod(p.shape.as_list()) for p in param])
			for p in param:
				numel = np.prod(p.shape.as_list())
				tensors.append(tf.reshape(vector[offset: offset+numel], p.shape))
				offset += numel

			assert offset == num_total_param
			return tensors

	def calc_norm(self, v):
		# default: frobenius norm
		if isinstance(v, list):
			norm = 0.
			for p in v:
				norm = norm + tf.norm(tensor=p)**2
			return norm**0.5
		else:
			return tf.norm(tensor=v)

	def _ops_in_minibatch(self):
		"""
		Define operations that will be used in method minibatch
		Vectorization is already a deep copy operation.
		Before using newton method, loss needs to be summed over training samples
		to make results consistent.
		"""

		def cal_loss():
			return tf.compat.v1.assign(self.f, self.f + self.loss)

		def cal_lossgrad():
			update_f = tf.compat.v1.assign(self.f, self.f + self.loss)

			grad = tf.gradients(ys=self.loss, xs=self.param)
			grad = self.vectorize(grad)
			update_grad = tf.compat.v1.assign(self.g, self.g + grad)

			return tf.group(*[update_f, update_grad])

		def cal_lossGv():
			v = self.inverse_vectorize(self.v, self.param)
			Gv = Gauss_Newton_vec(self.outputs, self.loss, self.param, v)
			Gv = self.vectorize(Gv)
			return tf.compat.v1.assign(self.Gv, self.Gv + Gv) 

		# add regularization term to loss, gradient and Gv and further average over batches 
		def add_reg_avg_loss():
			model_weight = self.vectorize(self.param)
			reg = (self.calc_norm(model_weight))**2
			reg = 1.0/(2*self.config.C) * reg
			return tf.compat.v1.assign(self.f, reg + self.f/self.config.num_data)

		def add_reg_avg_lossgrad():
			model_weight = self.vectorize(self.param)
			reg_grad = model_weight/self.config.C
			return tf.compat.v1.assign(self.g, reg_grad + self.g/self.config.num_data)

		def add_reg_avg_lossGv():
			return tf.compat.v1.assign(self.Gv, (self._lambda + 1/self.config.C)*self.v
			 + self.Gv/self.config.GNsize) 

		# zero out loss, grad and Gv 
		def zero_loss():
			return tf.compat.v1.assign(self.f, tf.zeros_like(self.f))
		def zero_grad():
			return tf.compat.v1.assign(self.g, tf.zeros_like(self.g))
		def zero_Gv():
			return tf.compat.v1.assign(self.Gv, tf.zeros_like(self.Gv))

		return (cal_loss(), cal_lossgrad(), cal_lossGv(),
				add_reg_avg_loss(), add_reg_avg_lossgrad(), add_reg_avg_lossGv(),
				zero_loss(), zero_grad(), zero_Gv())

	def minibatch(self, data_batch, place_holder_x, place_holder_y, mode):
		"""
		A function to evaluate either function value, global gradient or sub-sampled Gv
		"""
		if mode not in ('funonly', 'fungrad', 'Gv'):
			raise ValueError('Unknown mode other than funonly & fungrad & Gv!')

		inputs, labels = data_batch
		num_data = labels.shape[0]
		num_segment = math.ceil(num_data/self.config.bsize)
		x, y = place_holder_x, place_holder_y

		# before estimation starts, need to zero out f, grad and Gv according to the mode

		if mode == 'funonly':
			assert num_data == self.config.num_data
			assert num_segment == self.num_grad_segment
			self.sess.run(self.zero_loss)
		elif mode == 'fungrad':
			assert num_data == self.config.num_data
			assert num_segment == self.num_grad_segment
			self.sess.run([self.zero_loss, self.zero_grad])
		else:
			assert num_data == self.config.GNsize
			assert num_segment == self.num_Gv_segment
			self.sess.run(self.zero_Gv)

		for i in range(num_segment):
			
			load_time = time.time()
			idx = np.arange(i * self.config.bsize, min((i+1) * self.config.bsize, num_data))
			batch_input = inputs[idx]
			batch_labels = labels[idx]
			batch_input = np.ascontiguousarray(batch_input)
			batch_labels = np.ascontiguousarray(batch_labels)
			self.config.elapsed_time += time.time() - load_time

			if mode == 'funonly':

				self.sess.run(self.cal_loss, feed_dict={
							x: batch_input, 
							y: batch_labels,})

			elif mode == 'fungrad':
				
				self.sess.run(self.cal_lossgrad, feed_dict={
							x: batch_input, 
							y: batch_labels,})
				
			else:
				
				self.sess.run(self.cal_lossGv, feed_dict={
							x: batch_input, 
							y: batch_labels})

		# average over batches
		if mode == 'funonly':
			self.sess.run(self.add_reg_avg_loss)
		elif mode == 'fungrad':
			self.sess.run([self.add_reg_avg_loss, self.add_reg_avg_grad])
		else:
			self.sess.run(self.add_reg_avg_Gv, 
				feed_dict={self._lambda: self.config._lambda})


	def _update_model(self):
		update_model_ops = []
		x = self.inverse_vectorize(self.s, self.param)
		for i, p in enumerate(self.param):
			op = tf.compat.v1.assign(p, p + (self.alpha-self.old_alpha) * x[i])
			update_model_ops.append(op)
		return tf.group(*update_model_ops)

	def _init_cg_vars(self):
		init_ops = []

		init_r = tf.compat.v1.assign(self.r, -self.g)
		init_v = tf.compat.v1.assign(self.v, -self.g)
		init_s = tf.compat.v1.assign(self.s, tf.zeros_like(self.g))
		gnorm = self.calc_norm(self.g)
		init_rTr = tf.compat.v1.assign(self.rTr, gnorm**2)
		init_cgtol = tf.compat.v1.assign(self.cgtol, self.config.xi*gnorm)

		init_ops = [init_r, init_v, init_s, init_rTr, init_cgtol]

		return tf.group(*init_ops)

	def _CG(self):
		"""
		CG:
			define operations that will be used in method newton
		Same as the previous loss calculation,
		Gv has been summed over batches when samples were fed into Neural Network.
		"""

		def CG_ops():
			
			vGv = tf.tensordot(self.v, self.Gv, axes=1)

			alpha = self.rTr / vGv
			with tf.control_dependencies([alpha]):
				update_s = tf.compat.v1.assign(self.s, self.s + alpha * self.v, name='update_s_ops')
				update_r = tf.compat.v1.assign(self.r, self.r - alpha * self.Gv, name='update_r_ops')

				with tf.control_dependencies([update_s, update_r]):
					rnewTrnew = self.calc_norm(update_r)**2
					update_beta = tf.compat.v1.assign(self.beta, rnewTrnew / self.rTr)
					with tf.control_dependencies([update_beta]):
						update_rTr = tf.compat.v1.assign(self.rTr, rnewTrnew, name='update_rTr_ops')

			return tf.group(*[update_s, update_beta, update_rTr])

		def update_v():
			return tf.compat.v1.assign(self.v, self.r + self.beta*self.v, name='update_v')

		return (CG_ops(), update_v())


	def newton(self, full_batch, val_batch, saver, network, test_network=None):
		"""
		Conduct newton steps for training
		args:
			full_batch & val_batch: provide training set and validation set. The function will
				save the best model evaluted on validation set for future prediction.
			network: a tuple contains (x, y, loss, outputs).
			test_network: a tuple similar to argument network. If you use layers which behave differently
				in test phase such as batchnorm, a separate test_network is needed.
		return:
			None
		"""
		# check whether data is valid
		full_inputs, full_labels = full_batch
		assert full_inputs.shape[0] == full_labels.shape[0]

		if full_inputs.shape[0] != self.config.num_data:
			raise ValueError('The number of full batch inputs does not agree with the config argument.\
							This is important because global loss is averaged over those inputs')

		x, y, _, outputs = network

		tf.compat.v1.summary.scalar('loss', self.f)
		merged = tf.compat.v1.summary.merge_all()
		train_writer = tf.compat.v1.summary.FileWriter('./summary/train', self.sess.graph)

		print(self.config.args)
		if not self.config.screen_log_only:
			log_file = open(self.config.log_file, 'w')
			print(self.config.args, file=log_file)
		
		self.minibatch(full_batch, x, y, mode='fungrad')
		f = self.sess.run(self.f)
		output_str = 'initial f: {:.3f}'.format(f)
		print(output_str)
		if not self.config.screen_log_only:
			print(output_str, file=log_file)
		
		best_acc = 0.0

		total_running_time = 0.0
		self.config.elapsed_time = 0.0
		total_CG = 0
		
		for k in range(self.config.iter_max):

			# randomly select the batch for Gv estimation
			idx = np.random.choice(np.arange(0, full_labels.shape[0]),
					size=self.config.GNsize, replace=False)

			mini_inputs = full_inputs[idx]
			mini_labels = full_labels[idx]

			start = time.time()

			self.sess.run(self.init_cg_vars)
			cgtol = self.sess.run(self.cgtol)

			avg_cg_time = 0.0
			for CGiter in range(1, self.config.CGmax+1):
				
				cg_time = time.time()
				self.minibatch((mini_inputs, mini_labels), x, y, mode='Gv')
				avg_cg_time += time.time() - cg_time
				
				self.sess.run(self.CG)

				rnewTrnew = self.sess.run(self.rTr)
				
				if rnewTrnew**0.5 <= cgtol or CGiter == self.config.CGmax:
					break

				self.sess.run(self.update_v)

			print('Avg time per Gv iteration: {:.5f} s\r\n'.format(avg_cg_time/CGiter))

			gs, sGs = self.sess.run([self.update_gs, self.update_sGs], feed_dict={
					self._lambda: self.config._lambda
				})
			
			# line_search
			f_old = f
			alpha = 1
			while True:

				old_alpha = 0 if alpha == 1 else alpha/0.5
				
				self.sess.run(self.update_model, feed_dict={
					self.alpha:alpha, self.old_alpha:old_alpha
					})

				prered = alpha*gs + (alpha**2)*sGs

				self.minibatch(full_batch, x, y, mode='funonly')
				f = self.sess.run(self.f)

				actred = f - f_old

				if actred <= self.config.eta*alpha*gs:
					break

				alpha *= 0.5

			# update lambda
			ratio = actred / prered
			if ratio < 0.25:
				self.config._lambda *= self.config.boost
			elif ratio >= 0.75:
				self.config._lambda *= self.config.drop

			self.minibatch(full_batch, x, y, mode='fungrad')
			f = self.sess.run(self.f)

			gnorm = self.sess.run(self.gnorm)

			summary = self.sess.run(merged)
			train_writer.add_summary(summary, k)

			# exclude data loading time for fair comparison
			end = time.time() 
			
			end = end - self.config.elapsed_time
			total_running_time += end-start

			self.config.elapsed_time = 0.0
			
			total_CG += CGiter

			output_str = '{}-iter f: {:.3f} |g|: {:.5f} alpha: {:.3e} ratio: {:.3f} lambda: {:.5f} #CG: {} actred: {:.5f} prered: {:.5f} time: {:.3f}'.\
							format(k, f, gnorm, alpha, actred/prered, self.config._lambda, CGiter, actred, prered, end-start)
			print(output_str)
			if not self.config.screen_log_only:
				print(output_str, file=log_file)

			if val_batch is not None:
				# Evaluate the performance after every Newton Step
				if test_network == None:
					val_loss, val_acc, _ = predict(
						self.sess, 
						network=(x, y, self.loss, outputs),
						test_batch=val_batch,
						bsize=self.config.bsize,
						)
				else:
					# A separat test network part has not been done...
					val_loss, val_acc, _ = predict(
						self.sess, 
						network=test_network,
						test_batch=val_batch,
						bsize=self.config.bsize
						)

				output_str = '\r\n {}-iter val_acc: {:.3f}% val_loss {:.3f}\r\n'.\
					format(k, val_acc*100, val_loss)
				print(output_str)
				if not self.config.screen_log_only:
					print(output_str, file=log_file)

				if val_acc > best_acc:
					best_acc = val_acc
					checkpoint_path = self.config.model_file
					save_path = saver.save(self.sess, checkpoint_path)
					print('Best model saved in {}\r\n'.format(save_path))

		if val_batch is None:
			checkpoint_path = self.config.model_file
			save_path = saver.save(self.sess, checkpoint_path)
			print('Model at the last iteration saved in {}\r\n'.format(save_path))
			output_str = 'total_#CG {} | total running time {:.3f}s'.format(total_CG, total_running_time)
		else:
			output_str = 'Final acc: {:.3f}% | best acc {:.3f}% | total_#CG {} | total running time {:.3f}s'.\
				format(val_acc*100, best_acc*100, total_CG, total_running_time)
		print(output_str)
		if not self.config.screen_log_only:
			print(output_str, file=log_file)
			log_file.close()

##Set Train Arguments##

In [12]:
if USE_HFO:
    # Arguments for HFO - PSSP dataset
    train_args = ("--optim NewtonCG --GNsize 4096 --C 0.01 --net CNN_4layers --bsize 1024 --iter_max 100 " +
              "--train_set " + TRAIN_FILE + " --val_set " + VALID_FILE + " --dim " + 
              str(HEIGHT) + " " + str(WIDTH) + " " + str(DEPTH)).split()
else:
    # Arguments for SGD - PSSP dataset
    train_args = ("--optim SGD --lr 0.05 --momentum 0.01 --C 0.01 --net CNN_4layers --bsize 1024 --epoch_max 1000 " +
              "--train_set " + TRAIN_FILE + " --val_set " + VALID_FILE + " --dim " +
              str(HEIGHT) + " " + str(WIDTH) + " " + str(DEPTH)).split()

##Declare Train Function##

In [13]:
# import pdb
# import numpy as np
# import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
# import time
# import math
# import argparse

# from net.net import CNN
# from newton_cg import newton_cg
# from utilities import read_data, predict, ConfigClass, normalize_and_reshape

def parse_args():
	parser = argparse.ArgumentParser(description='Newton method on DNN')
	parser.add_argument('--C', dest='C',
					  help='regularization term, or so-called weight decay where'+\
					  		'weight_decay = lr/(C*num_of_samples) in this implementation' ,
					  default=0.01, type=float)

	# Newton method arguments
	parser.add_argument('--GNsize', dest='GNsize',
					  help='number of samples for estimating Gauss-Newton matrix',
					  default=4096, type=int)
	parser.add_argument('--iter_max', dest='iter_max',
					  help='the maximal number of Newton iterations',
					  default=100, type=int)
	parser.add_argument('--xi', dest='xi',
					  help='the tolerance in the relative stopping condition for CG',
					  default=0.1, type=float)
	parser.add_argument('--drop', dest='drop',
					  help='the drop constants for the LM method',
					  default=2/3, type=float)
	parser.add_argument('--boost', dest='boost',
					  help='the boost constants for the LM method',
					  default=3/2, type=float)
	parser.add_argument('--eta', dest='eta',
					  help='the parameter for the line search stopping condition',
					  default=0.0001, type=float)
	parser.add_argument('--CGmax', dest='CGmax',
					  help='the maximal number of CG iterations',
					  default=250, type=int)
	parser.add_argument('--lambda', dest='_lambda',
					  help='the initial lambda for the LM method',
					  default=1, type=float)

	# SGD arguments
	parser.add_argument('--epoch_max', dest='epoch',
					  help='number of training epoch',
					  default=500, type=int)
	parser.add_argument('--lr', dest='lr',
					  help='learning rate',
					  default=0.01, type=float)
	parser.add_argument('--decay', dest='lr_decay',
					  help='learning rate decay over each mini-batch update',
					  default=0, type=float)
	parser.add_argument('--momentum', dest='momentum',
					  help='momentum of learning',
					  default=0, type=float)

	# Model training arguments
	parser.add_argument('--bsize', dest='bsize',
					  help='batch size to evaluate stochastic gradient, Gv, etc. Since the sampled data \
					  for computing Gauss-Newton matrix and etc. might not fit into memeory \
					  for one time, we will split the data into several segements and average\
					  over them.',
					  default=1024, type=int)
	parser.add_argument('--net', dest='net',
					  help='classifier type',
					  default='CNN_4layers', type=str)
	parser.add_argument('--train_set', dest='train_set',
					  help='provide the directory of .mat file for training',
					  default=None, type=str)
	parser.add_argument('--val_set', dest='val_set',
					  help='provide the directory of .mat file for validation',
					  default=None, type=str)
	parser.add_argument('--model', dest='model_file',
					  help='model saving address',
					  default='./saved_model/model.ckpt', type=str)
	parser.add_argument('--log', dest='log_file',
					  help='log saving directory',
					  default='./running_log/logger.log', type=str)
	parser.add_argument('--screen_log_only', dest='screen_log_only',
					  help='screen printing running log instead of storing it',
					  action='store_true')
	parser.add_argument('--optim', '-optim', 
					  help='which optimizer to use: SGD, Adam or NewtonCG',
					  default='NewtonCG', type=str)
	parser.add_argument('--loss', dest='loss', 
					  help='which loss function to use: MSELoss or CrossEntropy',
					  default='MSELoss', type=str)
	parser.add_argument('--dim', dest='dim', nargs='+', help='input dimension of data,'+\
						'shape must be:  height width num_channels',
					  default=[32, 32, 3], type=int)
	parser.add_argument('--seed', dest='seed', help='a nonnegative integer for \
						reproducibility', type=int)	 
	
	args = parser.parse_args(args=train_args)
	return args


args = parse_args()

def init_model(param):
	init_ops = []
	for p in param:
		if 'kernel' in p.name:
			weight = np.random.standard_normal(p.shape)* np.sqrt(2.0 / ((np.prod(p.get_shape().as_list()[:-1]))))
			opt = tf.compat.v1.assign(p, weight)
		elif 'bias' in p.name:
			zeros = np.zeros(p.shape)
			opt = tf.compat.v1.assign(p, zeros)
		init_ops.append(opt)
	return tf.group(*init_ops)

def gradient_trainer(config, sess, network, full_batch, val_batch, saver, test_network):
	x, y, loss, outputs,  = network
	
	global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
	learning_rate = tf.compat.v1.placeholder(tf.float32, shape=[], name='learning_rate')

	# Probably not a good way to add regularization.
	# Just to confirm the implementation is the same as MATLAB.
	reg = 0.0
	param = tf.compat.v1.trainable_variables()
	for p in param:
		reg = reg + tf.reduce_sum(input_tensor=tf.pow(p,2))
	reg_const = 1/(2*config.C)
	batch_size = tf.compat.v1.cast(tf.shape(x)[0], tf.float32)
	loss_with_reg = reg_const*reg + loss/batch_size

	if config.optim == 'SGD':
		optimizer = tf.compat.v1.train.MomentumOptimizer(
					learning_rate=learning_rate, 
					momentum=config.momentum).minimize(
					loss_with_reg, 
					global_step=global_step)
	elif config.optim == 'Adam':
		optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate,
								beta1=0.9,
								beta2=0.999,
								epsilon=1e-08).minimize(
								loss_with_reg, 
								global_step=global_step)

	train_inputs, train_labels = full_batch
	num_data = train_labels.shape[0]
	num_iters = math.ceil(num_data/config.bsize)

	print(config.args)
	if not config.screen_log_only:
		log_file = open(config.log_file, 'w')
		print(config.args, file=log_file)
	sess.run(tf.compat.v1.global_variables_initializer())
	

	print('-------------- initializing network by methods in He et al. (2015) --------------')
	param = tf.compat.v1.trainable_variables()
	sess.run(init_model(param))

	total_running_time = 0.0
	best_acc = 0.0
	lr = config.lr

	for epoch in range(0, args.epoch):
		
		loss_avg = 0.0
		start = time.time()

		for i in range(num_iters):
			
			load_time = time.time()
			# randomly select the batch
			idx = np.random.choice(np.arange(0, num_data), 
					size=config.bsize, replace=False)

			batch_input = train_inputs[idx]
			batch_labels = train_labels[idx]
			batch_input = np.ascontiguousarray(batch_input)
			batch_labels = np.ascontiguousarray(batch_labels)
			config.elapsed_time += time.time() - load_time

			step, _, batch_loss= sess.run(
				[global_step, optimizer, loss_with_reg],
				feed_dict = {x: batch_input, y: batch_labels, learning_rate: lr}
				)

			# print initial loss
			if epoch == 0 and i == 0:
				output_str = 'initial f (reg + avg. loss of 1st batch): {:.3f}'.format(batch_loss)
				print(output_str)
				if not config.screen_log_only:
					print(output_str, file=log_file)

			loss_avg = loss_avg + batch_loss
			# print log every 10% of the iterations
			if i % math.ceil(num_iters/10) == 0:
				end = time.time()
				output_str = 'Epoch {}: {}/{} | loss {:.4f} | lr {:.6} | elapsed time {:.3f}'\
					.format(epoch, i, num_iters, batch_loss , lr, end-start)
				print(output_str)
				if not config.screen_log_only:
					print(output_str, file=log_file)
			
			# adjust learning rate for SGD by inverse time decay
			if args.optim != 'Adam':
				lr = config.lr/(1 + args.lr_decay*step)

		# exclude data loading time for fair comparison
		epoch_end = time.time() - config.elapsed_time
		total_running_time += epoch_end - start
		config.elapsed_time = 0.0
		
		if val_batch is None:
			output_str = 'In epoch {} train loss: {:.3f} | epoch time {:.3f}'\
				.format(epoch, loss_avg/(i+1), epoch_end-start)			
		else:
			if test_network == None:
				val_loss, val_acc, _ = predict(
					sess, 
					network=(x, y, loss, outputs),
					test_batch=val_batch,
					bsize=config.bsize
					)
			else:
				# A separat test network part have been done...
				val_loss, val_acc, _ = predict(
					sess, 
					network=test_network,
					test_batch=val_batch,
					bsize=config.bsize
					)
			
			output_str = 'In epoch {} train loss: {:.3f} | val loss: {:.3f} | val accuracy: {:.3f}% | epoch time {:.3f}'\
				.format(epoch, loss_avg/(i+1), val_loss, val_acc*100, epoch_end-start)
		
			if val_acc > best_acc:
				best_acc = val_acc
				checkpoint_path = config.model_file 
				save_path = saver.save(sess, checkpoint_path)
				print('Saved best model in {}'.format(save_path))

		print(output_str)
		if not config.screen_log_only:
			print(output_str, file=log_file)

	if val_batch is None:
		checkpoint_path = config.model_file 
		save_path = saver.save(sess, checkpoint_path)
		print('Model at the last iteration saved in {}\r\n'.format(save_path))
		output_str = 'total running time {:.3f}s'.format(total_running_time)
	else:
		output_str = 'Final acc: {:.3f}% | best acc {:.3f}% | total running time {:.3f}s'\
			.format(val_acc*100, best_acc*100, total_running_time)
	
	print(output_str)
	if not config.screen_log_only:
		print(output_str, file=log_file)
		log_file.close()

def newton_trainer(config, sess, network, full_batch, val_batch, saver, test_network):

	_, _, loss, outputs = network
	newton_solver = newton_cg(config, sess, outputs, loss)
	sess.run(tf.compat.v1.global_variables_initializer())

	print('-------------- initializing network by methods in He et al. (2015) --------------')
	param = tf.compat.v1.trainable_variables()
	sess.run(init_model(param))
	newton_solver.newton(full_batch, val_batch, saver, network, test_network)


def train_model():
	full_batch, num_cls, label_enum = read_data(filename=args.train_set, dim=args.dim)
	
	if args.val_set is None:
		print('No validation set is provided. Will output model at the last iteration.')
		val_batch = None
	else:
		val_batch, _, _ = read_data(filename=args.val_set, dim=args.dim, label_enum=label_enum)

	num_data = full_batch[0].shape[0]
	
	config = ConfigClass(args, num_data, num_cls)

	if isinstance(config.seed, int):
		tf.compat.v1.random.set_random_seed(config.seed)
		np.random.seed(config.seed)

	if config.net in ('CNN_4layers', 'CNN_7layers', 'VGG11', 'VGG13', 'VGG16','VGG19'):
		x, y, outputs = CNN(config.net, num_cls, config.dim)
		test_network = None
	else:
		raise ValueError('Unrecognized training model')

	if config.loss == 'MSELoss':
		loss = tf.reduce_sum(input_tensor=tf.pow(outputs-y, 2))
	else:
		loss = tf.reduce_sum(input_tensor=tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=y))
	
	network = (x, y, loss, outputs)

	sess_config = tf.compat.v1.ConfigProto()
	sess_config.gpu_options.allow_growth = True

	with tf.compat.v1.Session(config=sess_config) as sess:
		
		full_batch[0], mean_tr = normalize_and_reshape(full_batch[0], dim=config.dim, mean_tr=None)
		if val_batch is not None:
			val_batch[0], _ = normalize_and_reshape(val_batch[0], dim=config.dim, mean_tr=mean_tr)

		param = tf.compat.v1.trainable_variables()

		mean_param = tf.compat.v1.get_variable(name='mean_tr', initializer=mean_tr, trainable=False, 
					validate_shape=True, use_resource=False)
		label_enum_var=tf.compat.v1.get_variable(name='label_enum', initializer=label_enum, trainable=False,
					validate_shape=True, use_resource=False)
		saver = tf.compat.v1.train.Saver(var_list=param+[mean_param])
		
		if config.optim in ('SGD', 'Adam'):
			gradient_trainer(
				config, sess, network, full_batch, val_batch, saver, test_network)
		elif config.optim == 'NewtonCG':
			newton_trainer(
				config, sess, network, full_batch, val_batch, saver, test_network=test_network)

## Train ##

In [14]:
train_model()

You choose not to specify a random seed.A different result is produced after each run.
Saving log to: ./running_log/logger.log
No mean of data provided! Normalize images by their own mean.
Normalize images according to the provided mean.
-------------- initializing network by methods in He et al. (2015) --------------
Namespace(C=0.01, CGmax=250, GNsize=4096, _lambda=1, boost=1.5, bsize=1024, dim=[32, 32, 1], drop=0.6666666666666666, epoch=500, eta=0.0001, iter_max=100, log_file='./running_log/logger.log', loss='MSELoss', lr=0.01, lr_decay=0, model_file='./saved_model/model.ckpt', momentum=0, net='CNN_4layers', optim='NewtonCG', screen_log_only=False, seed=None, train_set='/content/drive/MyDrive/Datasets/cb513_protbert_trainSet2.mat', val_set='/content/drive/MyDrive/Datasets/cb513_protbert_testSet2.mat', xi=0.1)
initial f: 1.257
Avg time per Gv iteration: 0.74604 s

0-iter f: 0.929 |g|: 2.60463 alpha: 1.000e+00 ratio: 0.974 lambda: 0.66667 #CG: 1 actred: -0.32868 prered: -0.33758 time:

## Predict ##

In [15]:
# Arguments for prediction PSSP dataset
pred_args = ("--bsize 1024 --valid_set " + VALID_FILE + " --train_set " + TRAIN_FILE + 
						 " --model ./saved_model/model.ckpt --dim " +
             str(HEIGHT) + " " + str(WIDTH) + " " + str(DEPTH)).split()

In [16]:
valid_f =  "/content/drive/MyDrive/Datasets/{0}_test_fold{1}.txt".format(dataset.lower(),str(fold)) # train set  
train_f = "/content/drive/MyDrive/Datasets/{0}_train_fold{1}.txt".format(dataset.lower(),str(fold)) # validation set
test_f = "/content/drive/MyDrive/Datasets/CASP13_3class.txt" # test set CASP13
print(valid_f)
print(train_f)
print(test_f)

/content/drive/MyDrive/Datasets/cb513_test_fold2.txt
/content/drive/MyDrive/Datasets/cb513_train_fold2.txt
/content/drive/MyDrive/Datasets/CASP13_3class.txt


In [17]:
VALID_PRED_FILE="pred_test_fold{0}.txt".format(fold)
TRAIN_PRED_FILE="pred_train_fold{0}.txt".format(fold)
TEST_PRED_FILE="pred_casp13_fold{0}.txt".format(fold)
print(VALID_PRED_FILE)
print(TRAIN_PRED_FILE)
print(TEST_PRED_FILE)

pred_test_fold2.txt
pred_train_fold2.txt
pred_casp13_fold2.txt


##Declare Predict Methods##

In [18]:
def create_output_pred(pred, origin_f, outFileName):
    labels = ['C','H','E']
    read_file = open(origin_f,"r")
    output_file = open(outFileName,"w")
    count =1 
    target_name =1
    target_secondary = 3
    counter = 0
    while True:
        line = read_file.readline()
        if not line:
            break
        if count == target_name:
            output_file.write(line)
            target_name+=3
        if count == target_secondary:
            output_file.write(line)
            target_secondary+=3
            line = line.replace("\n","") 
            prediction = ""
            for c in line:
                if (c!='!'):
                    prediction = prediction + labels[pred[counter]]
                    counter +=1
            output_file.write(prediction + "\n")        
        count+=1
        

In [19]:
# import tensorflow as tf 
# tf.compat.v1.disable_eager_execution()
# from utilities import predict, read_data, normalize_and_reshape
# from net.net import CNN
# import numpy as np 
# import argparse
# import pdb

def parse_args():
        parser = argparse.ArgumentParser(description='prediction')
        parser.add_argument('--test_set', dest='test_set',
                            help='provide the directory of .mat file for testing',
                            default=None, type=str)
        parser.add_argument('--valid_set', dest='valid_set',
                            help='provide the directory of .mat file for validation',
                            default=None, type=str)
        parser.add_argument('--train_set', dest='train_set',
                            help='provide the directory of .mat file for training',
                            default=None, type=str)
        parser.add_argument('--model', dest='model_file',
                            help='provide file storing network parameters, i.e. ./dir/model.ckpt',
                            default='./saved_model/model.ckpt', type=str)
        parser.add_argument('--bsize', dest='bsize',
                            help='batch size',
                            default=1024, type=int)
        parser.add_argument('--loss', dest='loss', 
                            help='which loss function to use: MSELoss or CrossEntropy',
                            default='MSELoss', type=str)
        parser.add_argument('--dim', dest='dim', nargs='+', help='input dimension of data,'+\
                            'shape must be:  height width num_channels',
                            default=[32, 32, 3], type=int)

        args = parser.parse_args(args=pred_args)
        return args

def predict_model():
        args = parse_args()

        sess_config = tf.compat.v1.ConfigProto()
        sess_config.gpu_options.allow_growth = True

        with tf.compat.v1.Session(config=sess_config) as sess:
                graph_address = args.model_file + '.meta'
                imported_graph = tf.compat.v1.train.import_meta_graph(graph_address)
                imported_graph.restore(sess, args.model_file)
                mean_param = [v for v in tf.compat.v1.global_variables() if 'mean_tr:0' in v.name][0]
                label_enum_var = [v for v in tf.compat.v1.global_variables() if 'label_enum:0' in v.name][0]

                sess.run(tf.compat.v1.variables_initializer([mean_param, label_enum_var]))
                mean_tr = sess.run(mean_param)
                label_enum = sess.run(label_enum_var)

                x = tf.compat.v1.get_default_graph().get_tensor_by_name('main_params/input_of_net:0')
                y = tf.compat.v1.get_default_graph().get_tensor_by_name('main_params/labels:0')
                outputs = tf.compat.v1.get_default_graph().get_tensor_by_name('output_of_net:0')

                if args.loss == 'MSELoss':
                        loss = tf.reduce_sum(input_tensor=tf.pow(outputs-y, 2))
                else:
                        loss = tf.reduce_sum(input_tensor=
                            tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=tf.stop_gradient(y)))
                
                network = (x, y, loss, outputs)

                if args.valid_set is not None:
                        valid_batch, num_cls, _ = read_data(args.valid_set, dim=args.dim, label_enum=label_enum)
                        valid_batch[0], _ = normalize_and_reshape(valid_batch[0], dim=args.dim, mean_tr=mean_tr)
            
                        avg_loss_valid, avg_acc_valid, results_valid = predict(sess, network, valid_batch, args.bsize)

                        # convert results back to the original labels
                        inverse_map = dict(zip(np.arange(num_cls), label_enum))
                        results_valid = np.expand_dims(results_valid, axis=1)
                        results_valid = np.apply_along_axis(lambda x: inverse_map[x[0]], axis=1, arr=results_valid)
                        create_output_pred(results_valid, valid_f, VALID_PRED_FILE)
                        print('In valid phase, average loss: {:.3f} | average accuracy: {:.3f}%'.\
                            format(avg_loss_valid, avg_acc_valid*100))
                
                if args.train_set is not None:
                        train_batch, num_cls, _ = read_data(args.train_set, dim=args.dim, label_enum=label_enum)
                        train_batch[0], _ = normalize_and_reshape(train_batch[0], dim=args.dim, mean_tr=mean_tr)

                        avg_loss_train, avg_acc_train, results_train = predict(sess, network, train_batch, args.bsize)
                        # convert results back to the original labels
                        inverse_map = dict(zip(np.arange(num_cls), label_enum))
                        results_train = np.expand_dims(results_train, axis=1)
                        results_train = np.apply_along_axis(lambda x: inverse_map[x[0]], axis=1, arr=results_train)
                        # create_output_pred(results, results_train)

                        create_output_pred(results_train, train_f, TRAIN_PRED_FILE)
                        print('In train phase, average loss: {:.3f} | average accuracy: {:.3f}%'.\
                            format(avg_loss_train, avg_acc_train*100))

                if args.test_set is not None:
                        test_batch, num_cls, _ = read_data(args.test_set, dim=args.dim, label_enum=label_enum)
                        test_batch[0], _ = normalize_and_reshape(test_batch[0], dim=args.dim, mean_tr=mean_tr)

                        avg_loss_test, avg_acc_test, results_test = predict(sess, network, test_batch, args.bsize)
                        # convert results back to the original labels
                        inverse_map = dict(zip(np.arange(num_cls), label_enum))
                        results_test = np.expand_dims(results_test, axis=1)
                        results_test = np.apply_along_axis(lambda x: inverse_map[x[0]], axis=1, arr=results_test)
                        # create_output_pred(results, results_train)

                        create_output_pred(results_test, test_f, TEST_PRED_FILE)
                        print('In test phase, average loss: {:.3f} | average accuracy: {:.3f}%'.\
                            format(avg_loss_test, avg_acc_test*100))
            

##Run Predict and Display output##

In [20]:
predict_model()

INFO:tensorflow:Restoring parameters from ./saved_model/model.ckpt
Normalize images according to the provided mean.
In valid phase, average loss: 0.341 | average accuracy: 78.131%
Normalize images according to the provided mean.
In train phase, average loss: 0.328 | average accuracy: 79.307%


In [21]:
# !head "$VALID_PRED_FILE"

In [22]:
# !head "$TRAIN_PRED_FILE"

## Check Test score on CASP13 ##

In [23]:
# Arguments for prediction PSSP dataset
pred_args = ("--bsize 1024 --test_set " + TEST_FILE + 
						 " --model ./saved_model/model.ckpt --dim " +
             str(HEIGHT) + " " + str(WIDTH) + " " + str(DEPTH)).split()

In [24]:
predict_model()

INFO:tensorflow:Restoring parameters from ./saved_model/model.ckpt
Normalize images according to the provided mean.
In test phase, average loss: 0.341 | average accuracy: 77.765%


In [25]:
# !head "$TEST_PRED_FILE"

In [26]:
!zip -r saved_model


zip error: Nothing to do! (saved_model.zip)
